# CSCI 3155 : Assignment 8

__Name__:

Kyle Moe

In [1]:
// TEST HELPER: Do not EDIT this cell please.
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 : Type Checking (100 points)

Write a function `typecheck` that typechecks a Lettuce program.

$$\texttt{typecheck} : Program \rightarrow Type$$ 

This function should take as input a lettuce _Program_, and return what _Type_ it is. If it doesn't typecheck it should return a special value _TypeError_. Use the Lettuce typing notes on moodle for details on how typechecking lettuce works.

The tests for this assigment are hidden, so you will need to write your own tests! Note that you are encouraged to collaborate and share your test ideas on Piazza. To get full points, your implementation needs to match the specification from the Lettuce typing notes on moodle.

In [2]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType( ty_a : Type, ty_r : Type) extends Type
case object TypeError extends Type

sealed trait Expr
case class  Const(n : Int) extends Expr
case class  Ident(s : String) extends Expr
case object True extends Expr
case object False extends Expr
case class  Plus( e1 : Expr, e2 : Expr) extends Expr
case class  Minus( e1 : Expr, e2 : Expr) extends Expr
case class  Mult( e1 : Expr, e2 : Expr) extends Expr
case class  Neg( e1 : Expr) extends Expr
case class  And( e1 : Expr, e2 : Expr) extends Expr
case class  Or( e1 : Expr, e2 : Expr) extends Expr
case class  Geq( e1 : Expr, e2 : Expr) extends Expr
case class  Eq( e1 : Expr, e2 : Expr) extends Expr
case class  ITE( ec : Expr, et : Expr, ef : Expr) extends Expr
case class  Let( name : String, ty_d : Type, ed : Expr, ei : Expr) extends Expr
case class  FunDef( arg : String, ty_a : Type, eb : Expr) extends Expr
case class  FunCall( ef : Expr, ea : Expr) extends Expr
case class  LetRec( f : String, ty_f : Type, x : String, ty_x : Type, eb : Expr, ei : Expr) extends Expr

sealed trait Program
case class Main(p : Expr) extends Program

defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined object TypeError
defined trait Expr
defined class Const
defined class Ident
defined object True
defined object False
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined class And
defined class Or
defined class Geq
defined class Eq
defined class ITE
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined trait Program
defined class Main

In [3]:
// By defining the following Context ADT we are able to store the type of each function and variable
sealed trait Context
case object Empty extends Context
case class Extend(key : String, t : Type, con : Context) extends Context

// Getter function for Context that determines if an Ident("X") corresponds to a known type
def get(c : Context, key : String) : Type = {
    c match {
        case Empty => TypeError 
        case Extend(k, t, con) => if (k == key) t else get(con, key)
    }
}

//Typecheck function for Main(Expr) kind of program
def typecheck(p : Program) : Type = {
    p match {
        //Ensure that our program is correctly constructed
        case Main(q) => {
            // We define a recursive helper function in order to make use of our defined Context
            def typeRec(i : Expr, con : Context) : Type = {
                i match {
                    case Const(n) => NumType 
                    case True => BoolType
                    case False => BoolType
                    case Ident(s) => get(con, s) // Depends on whether Identity has type in Context
                    
                    //Plus, Minus, and Mult are essentially the same and require that both args are
                    //NumType and if so returns NumType
                    case Plus(e1, e2) => {
                        if ((typeRec(e1, con) == NumType) && (typeRec(e2, con) == NumType)) NumType else TypeError
                    }
                    case Minus(e1, e2) => {
                        if ((typeRec(e1, con) == NumType) && (typeRec(e2, con) == NumType)) NumType else TypeError
                    }
                    case Mult(e1, e2) => {
                        if ((typeRec(e1, con) == NumType) && (typeRec(e2, con) == NumType)) NumType else TypeError
                    }
                    //Neg needs to be able to return return type if function or Expr type otherwise
                    case Neg(e) => typeRec(e, con) match {
                        case FunType(ta, tr) => tr
                        case BoolType => BoolType
                        case NumType => NumType
                        case TypeError => TypeError
                    }
                    // And and Or take BoolType args and returns BoolType
                    case And(e1, e2) => {
                        if ((typeRec(e1, con) == BoolType) && (typeRec(e2, con) == BoolType)) BoolType else TypeError
                    }
                    case Or(e1, e2) => {
                        if ((typeRec(e1, con) == BoolType) && (typeRec(e2, con) == BoolType)) BoolType else TypeError
                    }
                    //Geq and Eq take NumType and return BoolType
                    case Geq(e1, e2) => {
                        if ((typeRec(e1, con) == NumType) && (typeRec(e2, con) == NumType)) BoolType else TypeError
                    }
                    case Eq(e1, e2) => {
                        val t1 = typeRec(e1, con)
                        val t2 = typeRec(e2, con)
                        (t1, t2) match {
                            case (FunType(a, b), FunType(c, d)) => if (b == d) BoolType else TypeError
                            case (a, FunType(c, d)) => if (a == d) BoolType else TypeError
                            case (FunType(a, b), c) => if (b == c) BoolType else TypeError
                            case _ => if (t1 == t2) BoolType else TypeError
                        }
                        //if (typeRec(e1, con) == typeRec(e2, con)) BoolType else TypeError
                    }
                    // ITE requires that e1 is a BoolType conditional and that e2 and e3 are the same return type
                    case ITE(e1, e2, e3) => typeRec(e1, con) match {
                        case BoolType => {
                            val v2 = typeRec(e2, con)
                            val v3 = typeRec(e3, con)
                            if (v2 == v3) v2 else TypeError
                        }
                        case _ => TypeError
                    }
                    // We must evaluate the in expression so long as we know the let definition type in our Context
                    case Let(name, ty, ed, ei) => {
                        val newCon = Extend(name, ty, con)
                        typeRec(ei, newCon)
                    }
                    // FunDef returns FunType or Error but we must know argument type in Context to determine return type
                    case FunDef(a, tya, b) => {
                        val newCon = Extend(a, tya, con)
                        val tyb = typeRec(b, newCon)
                        tyb match {
                            case TypeError => TypeError
                            case _ => FunType(tya, tyb)
                        }
                    }
                    // When calling a function we want the function name to actually be associated with a
                    // FunType. From there ensure that argument type is appropriate as well
                    case FunCall(ef, ea) => {
                        val tyf = typeRec(ef, con)
                        val tya = typeRec(ea, con)
                        tyf match {
                            case FunType(ta, tr) => if (tya == ta) tr else TypeError
                            case _ => TypeError
                        }
                    }
                    // LetRec evaluates the function body in the context of the type of the function and its
                    // argument and from there ensures that the types of the function match the argument type
                    // and return type. The return type is defined by what returns from the in expression in
                    // when the function is included in the Context
                    case LetRec(f, tyf, a, tya, eb, ei) => {
                        val newCon1 = Extend(f, tyf, con)
                        val newCon2 = Extend(a, tya, newCon1)
                        val tyr = typeRec(eb, newCon2)
                        tyf match {
                            case FunType(ftya, ftyr) => {
                                if (ftya == tya && ftyr == tyr) {
                                    typeRec(ei, newCon1)
                                }else {
                                    TypeError
                                }
                                
                            }
                            case _ => TypeError
                        }
                        
                    }
                }
            }
            typeRec(q, Empty)
            
        }
        case _ => TypeError
    }
}

defined trait Context
defined object Empty
defined class Extend
defined function get
defined function typecheck

In [4]:
val p1 : Program = Main( Const(10) )
assert( typecheck(p1) == NumType, "test 1")
passed(20)
println("Hidden Tests : up to 80 points remaining")


*** Tests Passed (20 points) ***
Hidden Tests : up to 80 points remaining


p1: Program = Main(Const(10))

In [5]:
//Checks basic True/False
val p1 : Program = Main(True)
val p2 : Program = Main(False)
assert(typecheck(p1) == BoolType)
assert(typecheck(p2) == BoolType)

p1: Program = Main(True)
p2: Program = Main(False)

In [6]:
//Checks Plus
val p1 : Program = Main(Plus(Const(2), Const(-5)))
val p2 : Program = Main(Plus(Const(2), True))
val p3 : Program = Main(Plus(False, True))
assert(typecheck(p1) == NumType)
assert(typecheck(p2) == TypeError)
assert(typecheck(p3) == TypeError)

p1: Program = Main(Plus(Const(2), Const(-5)))
p2: Program = Main(Plus(Const(2), True))
p3: Program = Main(Plus(False, True))

In [7]:
//Checks Minus
val p1 : Program = Main(Minus(Const(2), Const(-5)))
val p2 : Program = Main(Minus(Const(2), True))
val p3 : Program = Main(Minus(False, True))
assert(typecheck(p1) == NumType)
assert(typecheck(p2) == TypeError)
assert(typecheck(p3) == TypeError)

p1: Program = Main(Minus(Const(2), Const(-5)))
p2: Program = Main(Minus(Const(2), True))
p3: Program = Main(Minus(False, True))

In [8]:
//Checks Mult
val p1 : Program = Main(Mult(Const(2), Const(-5)))
val p2 : Program = Main(Mult(Const(2), True))
val p3 : Program = Main(Mult(False, True))
assert(typecheck(p1) == NumType)
assert(typecheck(p2) == TypeError)
assert(typecheck(p3) == TypeError)

p1: Program = Main(Mult(Const(2), Const(-5)))
p2: Program = Main(Mult(Const(2), True))
p3: Program = Main(Mult(False, True))

In [9]:
//Checks Neg for NumType and BoolType
val p1 : Program = Main(Neg(Const(4)))
val p2 : Program = Main(Neg(False))
val p3 : Program = Main(Neg(Let( "f", FunType(NumType, NumType),
                     FunDef("x", NumType, Plus(Const(1), Ident("x"))),
                     FunCall( Ident("f"), Const(2)))))
assert(typecheck(p1) == NumType)
assert(typecheck(p2) == BoolType)
assert(typecheck(p3) == NumType)

p1: Program = Main(Neg(Const(4)))
p2: Program = Main(Neg(False))
p3: Program = Main(
  Neg(
    Let(
      "f",
      FunType(NumType, NumType),
      FunDef("x", NumType, Plus(Const(1), Ident("x"))),
      FunCall(Ident("f"), Const(2))
    )
  )
)

In [10]:
// Test ITE under multiple conditions as well as simple And and Or conditionals
val p1 : Program = Main(ITE(Geq(Const(1), Const(3)), True, False))
val p2 : Program = Main(ITE(Geq(Const(1), Const(3)), Const(0), False))
val p3 : Program = Main(ITE(Mult(Const(1), Const(4)), Const(0), Const(1)))
val p4 : Program = Main(And(Neg(False), True))
val p5 : Program = Main(Or(Neg(Const(-1)), True))
val p6 : Program = Main(ITE(False, Const(2), Mult(Const(3), Const(4))))

assert(typecheck(p1) == BoolType)
assert(typecheck(p2) == TypeError)
assert(typecheck(p3) == TypeError)
assert(typecheck(p4) == BoolType)
assert(typecheck(p5) == TypeError)
assert(typecheck(p6) == NumType)

p1: Program = Main(ITE(Geq(Const(1), Const(3)), True, False))
p2: Program = Main(ITE(Geq(Const(1), Const(3)), Const(0), False))
p3: Program = Main(ITE(Mult(Const(1), Const(4)), Const(0), Const(1)))
p4: Program = Main(And(Neg(False), True))
p5: Program = Main(Or(Neg(Const(-1)), True))
p6: Program = Main(ITE(False, Const(2), Mult(Const(3), Const(4))))

In [11]:
// Checks Let, FunDef, and FunCall
val p1 : Program = Main(Let( "f", FunType(NumType, NumType),
                     FunDef("x", NumType, Plus(Const(1), Ident("x"))),
                     FunCall( Ident("f"), Const(2))))
val p2 : Program = Main(Let("f", FunType(NumType, FunType(NumType, NumType)),
                   FunDef("x", NumType,
                     FunDef("y", NumType,
                       Plus(Ident("x"),Ident("y")))),
                   FunCall(FunCall(Ident("f"), Const(2)), Const(3))
                   ))
val p3 : Program = Main(Let("f", FunType(NumType, FunType(NumType, NumType)),
                   FunDef("x", NumType,
                     FunDef("y", NumType,
                       Plus(Ident("x"),Ident("y")))),
                   FunCall(FunCall(Ident("f"), False), Const(3))
                   ))
assert(typecheck(p1) == NumType)
assert(typecheck(p2) == NumType)
assert(typecheck(p3) == TypeError)

p1: Program = Main(
  Let(
    "f",
    FunType(NumType, NumType),
    FunDef("x", NumType, Plus(Const(1), Ident("x"))),
    FunCall(Ident("f"), Const(2))
  )
)
p2: Program = Main(
  Let(
    "f",
    FunType(NumType, FunType(NumType, NumType)),
    FunDef("x", NumType, FunDef("y", NumType, Plus(Ident("x"), Ident("y")))),
    FunCall(FunCall(Ident("f"), Const(2)), Const(3))
  )
)
p3: Program = Main(
  Let(
    "f",
    FunType(NumType, FunType(NumType, NumType)),
    FunDef("x", NumType, FunDef("y", NumType, Plus(Ident("x"), Ident("y")))),
    FunCall(FunCall(Ident("f"), False), Const(3))
  )
)

In [12]:
// Additional FunDef checks for lambda functions
val p1 = Main( FunDef("g", NumType, Let("h", NumType, Const(1), Mult(Ident("g"), Ident("h")))) )
val p2 = Main( FunDef("f", NumType, Eq( Ident("f"), Ident("1"))) )
val p3 = Main( FunDef("e", NumType, Mult( Ident("e"), Ident("e"))) )
assert(typecheck(p1) == FunType(NumType, NumType))
assert(typecheck(p2) == TypeError)
assert(typecheck(p3) == FunType(NumType, NumType))

p1: Main = Main(
  FunDef(
    "g",
    NumType,
    Let("h", NumType, Const(1), Mult(Ident("g"), Ident("h")))
  )
)
p2: Main = Main(FunDef("f", NumType, Eq(Ident("f"), Ident("1"))))
p3: Main = Main(FunDef("e", NumType, Mult(Ident("e"), Ident("e"))))

In [13]:
//Old Test -- Has FunDef within LetRec. The FunDef induces a typing error.
val p2 = Main(LetRec("fact", FunType(NumType, NumType), "x", NumType,
                        FunDef("x", NumType, 
                              ITE(Geq(Const(1), Ident("x")), 
                                 Const(1),
                                 Mult(Ident("x"), 
                                     FunCall(Ident("fact"),  
                                            Minus(Ident("x"), Const(1)))))),
                        FunCall(Ident("fact"), Const(5))))
// Young, sexy test without the redundant definition of "x" as NumType
val p1 = Main(LetRec("fact", FunType(NumType, NumType), "x", NumType,
                              ITE(Geq(Const(1), Ident("x")), 
                                 Const(1),
                                 Mult(Ident("x"), 
                                     FunCall(Ident("fact"),  
                                            Minus(Ident("x"), Const(1))))),
                        FunCall(Ident("fact"), Const(5))))
assert(typecheck(p1) == NumType)
typecheck(p2)

p2: Main = Main(
  LetRec(
    "fact",
    FunType(NumType, NumType),
    "x",
    NumType,
    FunDef(
      "x",
      NumType,
      ITE(
        Geq(Const(1), Ident("x")),
        Const(1),
        Mult(Ident("x"), FunCall(Ident("fact"), Minus(Ident("x"), Const(1))))
      )
    ),
    FunCall(Ident("fact"), Const(5))
  )
)
p1: Main = Main(
  LetRec(
    "fact",
    FunType(NumType, NumType),
    "x",
    NumType,
    ITE(
      Geq(Const(1), Ident("x")),
      Const(1),
      Mult(Ident("x"), FunCall(Ident("fact"), Minus(Ident("x"), Const(1))))
    ),
    FunCall(Ident("fact"), Const(5))
  )
)
res12_3: Type = TypeError

In [69]:
println("Hidden test")

Hidden test


In [15]:
println("Hidden test")

Hidden test


In [16]:
println("Hidden test")

Hidden test


In [17]:
println("Hidden test")

Hidden test


In [18]:
println("Hidden test")

Hidden test


In [19]:
println("Hidden test")

Hidden test


In [20]:
println("Hidden test")

Hidden test


In [21]:
println("Hidden test")

Hidden test


In [22]:
println("Hidden test")

Hidden test


In [23]:
println("Hidden test")

Hidden test


In [24]:
println("Hidden test")

Hidden test


In [25]:
println("Hidden test")

Hidden test


In [26]:
println("Hidden test")

Hidden test


In [27]:
println("Hidden test")

Hidden test


In [28]:
println("Hidden test")

Hidden test


In [29]:
println("Hidden test")

Hidden test


In [30]:
println("Hidden test")

Hidden test


In [31]:
println("Hidden test")

Hidden test


In [32]:
println("Hidden test")

Hidden test


In [33]:
println("Hidden test")

Hidden test


In [34]:
println("Hidden test")

Hidden test


In [35]:
println("Hidden test")

Hidden test


In [36]:
println("Hidden test")

Hidden test


In [37]:
println("Hidden test")

Hidden test


In [38]:
println("Hidden test")

Hidden test


In [39]:
println("Hidden test")

Hidden test


In [40]:
println("Hidden test")

Hidden test


In [41]:
println("Hidden test")

Hidden test


In [42]:
println("Hidden test")

Hidden test


In [43]:
println("Hidden test")

Hidden test


In [44]:
println("Hidden test")

Hidden test


In [45]:
println("Hidden test")

Hidden test


In [46]:
println("Hidden test")

Hidden test
